In [11]:
from charm.toolbox.pairinggroup import PairingGroup, ZR, G1, G2, pair
from charm.core.engine.util import objectToBytes
from charm.toolbox.IBSig import IBSig
from charm.schemes.pksig.pksig_bls04 import BLS01
import sys

In [12]:
import random
# Create a list of words
words = ["apple", "banana", "cherry", "dog", "elephant", "fish", "grape", "house", "ice cream", "juice", "kiwi", "lemon", "mango", "nut", "orange", "pear", "pineapple", "quince", "raspberry", "strawberry", "tomato", "umbrella", "violet", "watermelon", "x-ray", "yacht", "zebra"]

In [13]:
# Building a custom BLS group with required functionalities 
class CustomBLS01(BLS01):
    def __init__(self, groupObj):
        super().__init__(groupObj)  
        self.group = groupObj  
    def dump(self, obj):
        return objectToBytes(obj, self.group)

    def keygen(self, secparam=None, g=None):
        if g is None:
            g = self.group.random(G2)
        x = self.group.random()
        g_x = g ** x
        pk = {'g^x': g_x, 'g': g, 'identity': str(g_x), 'secparam': secparam}
        sk = {'x': x}
        return (pk, sk)

    def sign(self, x, message):
        M = self.dump(message)
        return self.group.hash(M, G1) ** x

    def verify(self, pk, sig, message):
        M = self.dump(message)
        h = self.group.hash(M, G1)
        if pair(sig, pk['g']) == pair(h, pk['g^x']):
            return True  
        return False 
    def aggregate(self, signatures):
        # Initialize the aggregated signature with 'None'. This will be updated with the first signature.
        aggregated_signature = None
        
        # Iterate over all the signatures in the dictionary
        for _, sig in signatures.items():
            # If the aggregated signature is None, this is the first signature, so we initialize it.
            if aggregated_signature is None:
                aggregated_signature = sig
            else:
                # Otherwise, we aggregate by adding the current signature to the aggregated signature.
                aggregated_signature = aggregated_signature + sig
                
        return aggregated_signature

    def aggregate_verify(self, agg_sign, pk, messages, user_count , generator):
        combined_rhs = None
        lhs = pair(agg_sign, generator)
        # print("\n",lhs)
        # rhs = {}
        for i in range(user_count):
            M = self.dump(messages[i])
            h = self.group.hash(M, G1)
            rhs = pair(h,pk[i]['g^x'])
            if combined_rhs is None:
                combined_rhs = rhs
            else:
                combined_rhs = combined_rhs * rhs
        # print("rhs",combined_rhs)
        if lhs==combined_rhs:
            return True
        else:
            return False

In [10]:
#initialize  group
groupObj = PairingGroup('MNT224')
bls_custom = CustomBLS01(groupObj)

#users and messages
message_count = user_count = 3
message={}

# random words 
for i in range(message_count):
    message[i] = random.choice(words)

#generate public and secret keys with a generator
pk_a, sk_a = bls_custom.keygen()
generator =pk_a['g']
pk_b, sk_b = bls_custom.keygen(g=generator)
pk_c, sk_c = bls_custom.keygen(g=generator)



#sign the respective messages
a_sig = bls_custom.sign(sk_a['x'], message[0])
b_sig = bls_custom.sign(sk_b['x'], message[1])
c_sig = bls_custom.sign(sk_c['x'], message[2])

#add signatures & Public Keys into Dictonary
signatures={}
signatures[0] = a_sig
signatures[1] = b_sig
signatures[2] = c_sig

pk = {}
pk[0] = pk_a
pk[1] = pk_b
pk[2] = pk_c

#aggregate signatures
sig_agg = bls_custom.aggregate(signatures)

#verify the aggregated signature
print(bls_custom.aggregate_verify(sig_agg, pk, message, user_count,generator))




True
